In [2]:
import  numpy as np
import  pandas as pd
import  torch 
from    typing import List, Dict, Any, Tuple, Union
from    omegaconf import DictConfig, OmegaConf
from    pathlib import Path

from    rich import print as rprint
from    rich.markdown import Markdown

import  matplotlib.pyplot as plt 
import  seaborn as sns

import  logging 
import  tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import scipy.stats as scs

In [4]:
import  aistudio.core.io.filesystem as fs
from    aistudio.core             import Experiment, read_env, import_modules
from    aistudio.core.io.reader   import read_hydra, read_csv_to_pandas
from    aistudio.core.io.utils    import trsfrm_frame_camelcase_to_snakecase
from    aistudio.core.transform   import trsfrm_dt_features_tod, trsfrm_timestamp_to_dt
from    aistudio.datasets.dataset import DatasetTabular
from    aistudio.datasets.info    import InfoDateTime, InfoTabular, SchemaInfo
from    aistudio.datasets.catalog import Catalog 

# Experiment

In [5]:
expname = "catembeddings-torch"  
exp     = Experiment(root_path="./aistudio-core")
seed    = exp.seed_init()
exp_config:DictConfig  = exp.create(experiment_name=expname, tags=expname.split('-')).experiment
exp_yaml:str           = OmegaConf.to_yaml(exp_config)
env_config:dict        = read_env(exp.root_dir.joinpath(".env"))
output_dir:str         = Path(exp_config.artifacts.datasets) 

In [6]:
# define configuration
column_sel = 'volume'

# Catalog

In [7]:
datasets_root_uri = f"{exp.data_dir}/catalog/datasets"
dataset_uri = f"{exp.data_dir}/catalog/datasets/imdb"
dataset_uri = f"{exp.data_dir}/catalog/datasets/anthem"

cat = Catalog(datasets_root_uri)
cat_metrics = cat.read_catalog_metrics(cat.df_catalog)
rprint(cat_metrics)
display(cat.df_catalog)

{'num_files': 7, 'num_dbs': 4}

,db,table,filename,uri
0,anthem/claims,claims_may_2024,claims_may_2024.csv,aistudio-core/data/catalog/datasets/anthem/cla...
1,anthem/eob,eob_apr_28_204,eob_apr_28_204.pdf,aistudio-core/data/catalog/datasets/anthem/eob...
2,anthem/eob,eob_may_12_2024,eob_may_12_2024.pdf,aistudio-core/data/catalog/datasets/anthem/eob...
3,anthem/eob,eob_may_24_2024,eob_may_24_2024.pdf,aistudio-core/data/catalog/datasets/anthem/eob...
4,anthem/gna,gna-2024-ytd,gna-2024-ytd.csv,aistudio-core/data/catalog/datasets/anthem/gna...
5,imdb,directors,directors.csv,aistudio-core/data/catalog/datasets/imdb/direc...
6,imdb,movies,movies.csv,aistudio-core/data/catalog/datasets/imdb/movie...


# Dataset

In [8]:
# source  = cat.read_datasource_properties('anthem/gna', 'gna-2024-ytd')
uri = exp.catalog_dir.joinpath('datasets/anthem/gna/gna-2024-ytd.csv')
df_dat  = (
    read_csv_to_pandas(uri, index_col=0, parse_dates=True)
    .pipe(trsfrm_frame_camelcase_to_snakecase)
    .rename_axis('date')
)
dataset = DatasetTabular(df_dat)
dataset.data.head()

,requested_by,category,volume
date,,,
2020-01-01,Member Authorized Repqresentative,Clinical,144
2020-01-02,Provider NonContracted,Clinical,72
2020-01-03,HealthPlan,Administrative,129
2020-01-04,Anthem Associate,Clinical,108
2020-01-05,Regulatory Risk,Clinical,5


In [9]:
schema:pd.DataFrame = dataset.df_schema
dt_props:dict       = pd.DataFrame( [dataset.get_timespan()] )
display(schema)

,data_dtype,data_dtype_inferred,schema_dtype
requested_by,object,object,categorical
category,object,object,categorical
volume,int64,int64,numerical


In [10]:
# global level statistics 
display(dataset.df_sparsity)
display(dataset.numerical_stats)
display(dataset.cat_stats.T)
#dataset.cat_dist

,count,pct
volume,0,0.0


,kurtosis,max,mean,median,min,skew,std,variation
volume,-1.191,200.0,102.059,104.0,1.0,-0.053,57.178,0.56


,count,unique,top,freq
requested_by,1577,12,Provider,156
category,1698,2,Administrative,864


In [11]:
# Orignal Data Span 
dt_props:dict    = InfoDateTime.calc_dt_stats(dataset.data, col='date')
dt_span:dict     = InfoDateTime.calc_dt_timespan(dt_props['min_date'], dt_props['max_date'])
dt_span

{'days': 1697, 'quarters': 18.86, 'years': 4.65}

# Filters and Transforms

In [16]:
# filter for particular time period
start_dt, end_dt = '2022-01-01', '2024-07-31'
df_dat_dt_flt    = dataset.data.loc[start_dt:end_dt]
dt_props:dict    = InfoDateTime.calc_dt_stats(df_dat_dt_flt, col='date')
dt_span:dict     = InfoDateTime.calc_dt_timespan(dt_props['min_date'], dt_props['max_date'])
display(dt_span)
# particular time frame 
df_dat_tod = trsfrm_dt_features_tod(df_dat_dt_flt)
df_dat_tod.head()


{'days': 942, 'quarters': 10.47, 'years': 2.58}

,requested_by,category,volume,date,day,month,quarter,year
date,,,,,,,,
2022-01-01,Other,Clinical,41,2022-01-01,1,1,1,2022
2022-01-02,Other,Clinical,167,2022-01-02,2,1,1,2022
2022-01-03,Provider Authorized Representative,Clinical,2,2022-01-03,3,1,1,2022
2022-01-04,Member Authorized Repqresentative,Clinical,191,2022-01-04,4,1,1,2022
2022-01-05,Provider Authorized Representative,Administrative,29,2022-01-05,5,1,1,2022


In [35]:
# partition the data
train, test = train_test_split(df_dat_tod,  random_state=42, test_size=0.2)
train, val  = train_test_split(train, random_state=42, test_size=0.2)
print(f"Train Shape: {train.shape} | Val Shape: {val.shape} | Test Shape: {test.shape}")

Train Shape: (603, 8) | Val Shape: (151, 8) | Test Shape: (189, 8)


# Analysis

In [17]:
def aggregate(df:pd.DataFrame, agg_col:str, grp_cols:List[str], sort_cols:List[str]):
    return (
        df.groupby(grp_cols)[agg_col]
        .agg([np.sum, np.mean, np.median, np.std])
        .sort_values(by=sort_cols, ascending=True)
    )

# aggregate(df_dat_tod, 'volume', ['year', 'month', 'requested_by', 'category'], ['year', 'month', 'sum'])
# aggregate(df_dat_tod, 'volume', ['year', 'month', 'category'], ['year', 'month', 'sum'])
# aggregate(df_dat_tod, 'volume', ['year', 'month'], ['year', 'month', 'sum'])['sum']

In [18]:
# TBD: Annotate data with text 
# fig, (ax1, ax2) = plt.subplots(1,2, figure=(16,6))
# dataset.data['requested_by'].value_counts().plot(kind='barh', xlabel='count', ax=ax1)
# dataset.data['category'].value_counts().plot(kind='barh', xlabel='count', ax=ax2)
# plt.tight_layout()

In [19]:
# resampled data for the appropriate timeframe 
column_sel   = 'volume'
df_daily     = df_dat_tod
df_monthly   = df_daily.resample('M')[column_sel].sum().to_frame()
df_quarterly = df_daily.resample('Q')[column_sel].sum().to_frame()

## Trend: Rate of change over Time Period
- .diff, .pct_change for short term changes

In [20]:
def calc_trend(df:pd.DataFrame, col:str) -> pd.DataFrame:
    # calculate pct change: releative rate of change, so changes over time need to be normalized
    return  df[col].to_frame().assign(**{
        # calculate absolute difference: magnitude, for values that don't matter much in scale
        f"{col}_diff"       : lambda df_: df_[col].diff(),
        f"{col}_abs_diff"   : lambda df_: df_[f"{col}_diff"].apply(abs),
        f"{col}_pct(%)"     : lambda df_: (df_[col].pct_change() * 100).round(3),
        f"{col}_trend_dir"  : lambda df_: df_[f"{col}_diff"].apply(lambda x: 'Increasing' if x > 0 else ('Decreasing' if x < 0 else 'No Change'))
    })
 
df_daily_trend     = calc_trend(df_daily,     column_sel)
df_monthly_trend   = calc_trend(df_monthly,   column_sel)
df_quarterly_trend = calc_trend(df_quarterly, column_sel)

### Sample Sizing

In [32]:
def detect_sample_size(data, signifigance_level=0.05, power=0.8):
    variability     = np.std(data)
    trend_magnitude = np.abs(np.mean(data.diff()))

    # Z Scores 
    Z_alpha = scs.norm.ppf(1 - signifigance_level / 2)  # Two-tailed test ~ 1.96
    Z_beta  = scs.norm.ppf(power)                       # ~ 0.84
    # Heuristic estimate for sample size
    estimated_sample_size = ((variability / trend_magnitude) ** 2) * (Z_alpha + Z_beta) ** 2    
    return int(np.ceil(estimated_sample_size))

def sample_size_to_days(sample_size, frequency='daily'):
    """Convert sample size to the number of days based on the frequency of data collection."""
    # Map frequency to days
    frequency_mapping = {
        'daily':     1,
        'weekly':    7,
        'monthly':   30,  # Approximate month as 30 days
        'quarterly': 90,
        'annually':  360
    }
    days_per_sample = frequency_mapping.get(frequency.lower(), 1)
    days_required   = sample_size * days_per_sample
    return days_required

num_samples = detect_sample_size(df_monthly_trend['volume'])
sample_size_to_days(num_samples, frequency='monthly')

15744990

## Expected Value Baselines

In [52]:
# get baseline statistics
mu, std = ( df_monthly[column_sel].mean(), df_monthly[column_sel].std() )
print(f"mean={mu.round(2)}, std={std.round(2)}")

mean=3065.55, std=332.96


In [65]:
def calc_expectation(ds:pd.Series, window:int=5) -> pd.DataFrame:
    N     = window
    alpha = 2 / (N + 1)
    col   = ds.name 
    return ds.to_frame().assign(**{
        f"mean"     : lambda df_: df_[col].mean(),
        f"mva_{N}"  : lambda df_: df_[col].rolling(N).mean(),
        f"ewm_{N}"  : lambda df_: df_[col].ewm(span=N, adjust=False).mean()
    })

df_monthly_exp = calc_expectation( df_monthly[column_sel], window=3)
df_monthly_exp.head()

,volume,mean,mva_3,ewm_3
date,,,,
2022-01-31,2843,3065.548387,NaN,2843.0000
2022-02-28,2786,3065.548387,NaN,2814.5000
2022-03-31,3541,3065.548387,3056.666667,3177.7500
2022-04-30,3474,3065.548387,3267.000000,3325.8750
2022-05-31,3085,3065.548387,3366.666667,3205.4375


# Deviations and Anomalies

In [21]:
threshold  = 3
mu, std    = np.mean(dataset.data[column_sel]), np.std(dataset.data[column_sel])
gaussian   = scs.norm(mu, std)
display(f"Dataset {column_sel} column -> mean: {mu}, std: {std}")

df_anomaly = dataset.data['volume'].to_frame().assign(
    abs_gaussian_delta   = lambda df_: df_[column_sel].apply(lambda xs: abs( gaussian.pdf(xs) - gaussian.pdf(mu) )  ),
    is_anomaly_guassian = lambda df_: df_['abs_gaussian_delta'].apply(lambda xs: xs > threshold).astype(int) 
)
df_anomaly.sort_values(by='volume', ascending=False)
#df_anomaly['is_anomaly_gussian'].value_counts()

'Dataset volume column -> mean: 102.05948174322732, std: 57.17817188930601'

,volume,abs_gaussian_delta,is_anomaly_guassian
date,,,
2023-10-07,200,0.005368,0
2023-08-28,200,0.005368,0
2024-07-21,200,0.005368,0
2023-01-27,200,0.005368,0
2020-11-12,200,0.005368,0
...,...,...,...
2021-09-11,1,0.005514,0
2021-04-04,1,0.005514,0
2022-11-06,1,0.005514,0
